# Introduction
Voter Analysis exploration

## Imports
Import libraries and write settings here.

In [333]:
from fastai.tabular import *

# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 60
pd.options.display.max_rows = 60

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')



# Data Import and Cleaning

In [334]:
#find the data
path = './data/'
df = pd.read_csv('./data/ccesplus.csv',encoding = "ISO-8859-1")

In [335]:
#get rid of spaces and caps in column names (if any)
df.columns = [col.replace(' ','_').lower() for col in df.columns]

In [336]:
# df.head()

# df.columns

# df.describe()

#any duplicate rows?
df.duplicated().sum()

0

In [337]:
#lets see how many columns, how many have no and some missing values
print (f"Total columns={len(df.columns)}")
print(f"Total columns with no Nans= {len(df.columns[~df.isnull().any()])}") 
print(f"Total columns with Nans= {len(df.columns[df.isnull().any()])}") #add ~ to get columns with no missing values

Total columns=866
Total columns with no Nans= 268
Total columns with Nans= 598


The columns with no NaNs (missing values) are the easiest to use since we dont have to impute missing values

## Get a subset of the rows of the data to work on if desired

In [338]:
#get first thousand rows
# df=df[:1000]

## See what the summary columns do
looks like the summaries (..sum) are just the average

In [339]:
columns=['cc.CC16_422c','cc.CC16_422d','cc.CC16_422e','cc.CC16_422f', 'cc.raceviewsum']
columns = [col.lower() for col in columns]
tmp_df=df[columns]
tmp_df.head()

,cc.cc16_422c,cc.cc16_422d,cc.cc16_422e,cc.cc16_422f,cc.raceviewsum
0,1.0,2.0,2.0,2.0,1.75
1,1.0,3.0,1.0,2.0,1.75
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,1.0,1.0,1.00


## Get the columns of interest

In [340]:
#contains all the columns that original logits were run on
#use these to start to compare neural network solution
columns = pd.read_excel('./data/results cces.xlsx',encoding = "ISO-8859-1", skiprows=3)
columns = list(columns.iloc[:,0])

#get the dependant variable(s)
# columns_dep_var= ['cc.TrumpGEVote','cc.TrumpPVote', 'cc.vote12.gop']
columns_dep_var= ['cc.trumpgevote']

#combine
columns.extend(columns_dep_var)

#strip rubbish
columns = [col.replace(' ','_').lower() for col in columns]

## Create dataframe for model training

In [341]:
#select out the columns of interest
df_s=df[columns]

In [342]:
len(df_s.iloc[0])

32

In [343]:
# how many unique values there are per column, 
# use that to guide which columns are categorical and which are continuous
#pick largest value that looks continuous, for instance raceviewsum=17 
#(but see the docs many of theses fields are floats which meand fastai ignores cardinality)
for col in df_s.columns:
    print(f"{col}:{df[col].nunique()}")

cc.blackdum:2
econ.mhi:2646
cc.sex:2
cc.maxeduc.4yr:2
econ.hhpov.p:1030
mort.ucd.despair.disc95.pdpy:2660
cc.union:2
cc.age:80
demo.popdense:2668
cc.emp.nojob:2
rustpc:2437
taa.wrks.disc95.pcpy:2120
cc.faminc:16
crashpc:2626
cc.newsint:5
cc.isimmigrant:2
cc.catholic:2
cc.cc16_305_2:2
job.uer:761
cc.child18:2
cc.evanprot:2
cc.whitedum:2
cc.religiosity:15
cc.cc16_351b:2
cc.cc16_307:4
cc.immviewsum:5
cc.cc16_304:5
cc.inddum:2
cc.ideo7:7
cc.raceviewsum:17
cc.repdum:2
cc.trumpgevote:2


In [344]:
#what types are the columns
df_s.dtypes

cc.blackdum                       int64
econ.mhi                        float64
cc.sex                            int64
cc.maxeduc.4yr                    int64
econ.hhpov.p                    float64
mort.ucd.despair.disc95.pdpy    float64
cc.union                        float64
cc.age                            int64
demo.popdense                   float64
cc.emp.nojob                      int64
rustpc                          float64
taa.wrks.disc95.pcpy            float64
cc.faminc                       float64
crashpc                         float64
cc.newsint                      float64
cc.isimmigrant                    int64
cc.catholic                     float64
cc.cc16_305_2                     int64
job.uer                         float64
cc.child18                      float64
cc.evanprot                     float64
cc.whitedum                       int64
cc.religiosity                  float64
cc.cc16_351b                    float64
cc.cc16_307                     float64


In [345]:
#the percentage of NaNs in each column, note the large percentage of missing values in the bottom columns
#I would guess that cc.faminc and cc.raceviewsum are critical
df_s.isna().sum().sort_values(ascending = True)/len(df)

cc.blackdum                     0.000000
cc.immviewsum                   0.000000
cc.whitedum                     0.000000
cc.cc16_305_2                   0.000000
cc.emp.nojob                    0.000000
cc.age                          0.000000
cc.isimmigrant                  0.000000
cc.sex                          0.000000
cc.maxeduc.4yr                  0.000000
cc.evanprot                     0.000666
cc.newsint                      0.000820
cc.catholic                     0.001130
demo.popdense                   0.001300
econ.hhpov.p                    0.001300
econ.mhi                        0.001300
taa.wrks.disc95.pcpy            0.001300
job.uer                         0.001300
cc.cc16_307                     0.001517
mort.ucd.despair.disc95.pdpy    0.001548
cc.child18                      0.001827
cc.cc16_351b                    0.001950
cc.cc16_304                     0.002059
cc.union                        0.002136
crashpc                         0.002229
rustpc          

In [346]:
#BUMMER! the dependant variable has a lot of missing values (36.7%)
df_s['cc.trumpgevote'].unique()

array([nan,  1.,  0.])

## Remove NaN dependent variable rows
whaddaya going to do?  You have no way of knowing how these people voted

In [347]:
print(f"There are {len(df_s)} rows in df_s")

There are 64600 rows in df_s


In [348]:
#dump the NaN rows
df_s = df_s[pd.notnull(df_s['cc.trumpgevote'] )]

In [360]:
#is the dataset balanced?
df_s['cc.trumpgevote'].value_counts()

0    22136
1    18754
Name: cc.trumpgevote, dtype: int64

## Split out categorical and continuous variables
see if fastai can auto do it

In [349]:
#give fastai a shot at splitting cat and cont variables
res_cont, res_cat = cont_cat_split(df_s,max_card=18)

In [350]:
print(res_cont)
print(res_cat)

['econ.mhi', 'econ.hhpov.p', 'mort.ucd.despair.disc95.pdpy', 'cc.union', 'cc.age', 'demo.popdense', 'rustpc', 'taa.wrks.disc95.pcpy', 'cc.faminc', 'crashpc', 'cc.newsint', 'cc.catholic', 'job.uer', 'cc.child18', 'cc.evanprot', 'cc.religiosity', 'cc.cc16_351b', 'cc.cc16_307', 'cc.immviewsum', 'cc.cc16_304', 'cc.inddum', 'cc.ideo7', 'cc.raceviewsum', 'cc.repdum', 'cc.trumpgevote']
['cc.blackdum', 'cc.sex', 'cc.maxeduc.4yr', 'cc.emp.nojob', 'cc.isimmigrant', 'cc.cc16_305_2', 'cc.whitedum']


In [351]:
#remove the dependant variable
res_cont = [x for x in res_cont if x not in columns_dep_var]
res_cat = [x for x in res_cat if x not in columns_dep_var]

# Analysis/Modeling

## Create databunch

In [352]:
#convert trumpgevote to long (otherwise fit fails)
df_s['cc.trumpgevote'] = df_s['cc.trumpgevote'].astype('int64');

In [353]:
procs = [FillMissing, Categorify, Normalize]

In [354]:
test = TabularList.from_df(df_s.iloc[800:1000].copy(),  cat_names=res_cat, cont_names=res_cont)

In [355]:
data = (TabularList.from_df(df_s,  cat_names=res_cat, cont_names=res_cont, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=columns_dep_var)
                           .add_test(test)
                           .databunch())

In [274]:
data.show_batch(rows=50)

cc.blackdum,cc.sex,cc.maxeduc.4yr,cc.emp.nojob,cc.isimmigrant,cc.cc16_305_2,cc.whitedum,econ.mhi_na,econ.hhpov.p_na,mort.ucd.despair.disc95.pdpy_na,cc.union_na,demo.popdense_na,rustpc_na,taa.wrks.disc95.pcpy_na,cc.faminc_na,crashpc_na,cc.newsint_na,cc.catholic_na,job.uer_na,cc.child18_na,cc.evanprot_na,cc.religiosity_na,cc.cc16_351b_na,cc.cc16_307_na,cc.cc16_304_na,cc.inddum_na,cc.ideo7_na,cc.raceviewsum_na,cc.repdum_na,econ.mhi,econ.hhpov.p,mort.ucd.despair.disc95.pdpy,cc.union,cc.age,demo.popdense,rustpc,taa.wrks.disc95.pcpy,cc.faminc,crashpc,cc.newsint,cc.catholic,job.uer,cc.child18,cc.evanprot,cc.religiosity,cc.cc16_351b,cc.cc16_307,cc.immviewsum,cc.cc16_304,cc.inddum,cc.ideo7,cc.raceviewsum,cc.repdum,target
0,0,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.2472,-0.5797,1.1936,-0.6274,1.6631,-0.2065,0.1861,0.2584,-1.1857,-0.6411,-0.6584,1.8658,0.4034,-0.5497,-0.6189,-0.6898,-1.0144,0.8111,0.7259,1.0896,-0.5936,1.6117,1.9227,1.6338,1
0,0,1,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2.3739,-1.4261,-0.4741,-0.6274,-0.9717,-0.0800,-0.3577,-0.4489,0.4439,0.1223,1.3950,-0.5360,-0.6935,1.8192,1.6158,1.4875,-1.0144,0.8111,-1.3099,0.0282,-0.5936,1.0625,0.3846,-0.6121,1
0,0,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.9945,0.7905,0.2456,1.5938,1.3494,9.1503,-0.6255,-0.3077,2.0735,-0.5761,-0.6584,-0.5360,-0.3017,-0.5497,-0.6189,-1.5607,-1.0144,-0.4906,-1.3099,0.0282,-0.5936,-1.6832,-1.1536,-0.6121,0
0,1,1,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2.8512,-1.4812,-1.4292,-0.6274,-1.7245,-0.0280,-0.6189,-0.7886,0.1180,-0.2861,-0.6584,1.8658,-1.1422,-0.5497,-0.6189,-0.6898,0.9857,-0.4906,-1.3099,0.0282,-0.5936,-1.1340,-1.1536,-0.6121,0
0,1,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.8131,-0.1151,-0.2020,-0.6274,0.7848,-0.2624,-1.0497,-0.6079,2.0735,-0.7344,-0.6584,-0.5360,-1.2205,-0.5497,-0.6189,-0.6898,-1.0144,0.8111,-1.3099,0.0282,-0.5936,-1.1340,-1.4612,-0.6121,0
0,1,0,0,0,0,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,-0.0856,-0.7726,-0.2585,-0.6274,0.7848,-0.2915,-0.9331,-0.8311,-0.2079,-0.3369,1.3950,1.8658,-1.6764,-0.5497,-0.6189,0.8343,0.9857,0.8111,-0.6313,0.0282,1.6846,-0.0358,0.0769,-0.6121,1
0,1,1,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1.2027,-0.5482,0.0229,-0.6274,-1.6618,-0.0187,-0.3307,0.2154,-0.2079,-0.3629,1.3950,-0.5360,0.1256,-0.5497,-0.6189,-1.1252,-1.0144,0.8111,-1.3099,-1.0332,-0.5936,-1.1340,-0.8459,-0.6121,0
0,1,0,0,0,0,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.9223,-0.6072,-0.4366,-0.6274,1.6631,-0.2861,0.8219,0.4863,-0.2079,0.2116,1.3950,-0.5360,-0.8715,-0.5497,1.6158,1.2698,0.9857,-0.4906,0.7259,0.0282,1.6846,1.6117,1.6151,-0.6121,1
0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.3934,-1.1584,3.6015,1.5938,1.4121,-0.2987,0.3566,-0.0736,1.0957,-0.1942,-0.6584,-0.5360,0.1470,-0.5497,-0.6189,-1.5607,-1.0144,-0.4906,-1.3099,0.0282,1.6846,-0.5849,-1.4612,-0.6121,0
0,0,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2.7624,-1.6781,0.7209,-0.6274,0.0320,-0.2329,-0.8913,-0.8311,1.4217,-0.2649,-0.6584,-0.5360,-1.1992,-0.5497,1.6158,1.4875,0.9857,0.8111,0.7259,1.0896,1.6

## Create a learner

In [356]:
#create a learner
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

## Fit the data

In [359]:
learn.fit(epochs=5, lr=1e-3)

epoch,train_loss,valid_loss,accuracy
1,0.166088,0.211779,0.935000
2,0.168706,0.205853,0.925000
3,0.165994,0.222124,0.915000
4,0.172437,0.235784,0.910000
5,0.169955,0.237098,0.920000


# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here